### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
#reading/scraping the webpage using pandas
import pandas as pd
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [2]:
#intial shape of the dataframe
df.shape

(287, 3)

In [3]:
#indexing the 'Not assigned' Boroughs
indexNames = df[df['Borough']=='Not assigned'].index

In [4]:
#Dropping the Indexed 'Not assigned' Boroughs
df.drop(indexNames, inplace=True)

In [5]:
#shape after dropping 'Not assigned' Boroughs
df.shape

(210, 3)

In [6]:
#Grouping by Postcode
df=df.groupby(['Postcode', 'Borough']) ['Neighbourhood'].apply (list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')

In [7]:
#finding the 'Not assigned' Neighbourhood 
df.loc[df['Postcode']=='M9A']

,Postcode,Borough,Neighbourhood
46,M9A,Queen's Park,Not assigned


In [8]:
#replacing Neighbourhood value 'Not assigned' with Borough value
df.loc[(df.Postcode == 'M9A'),'Neighbourhood']=df['Borough']

### Replacing not assigned Neighbourhood 

In [9]:
#checking if the Neighbourhood value 'Not assigned' has updated with Borough value for Postcode M9A 
df.loc[df['Postcode']=='M9A']

,Postcode,Borough,Neighbourhood
46,M9A,Queen's Park,Queen's Park


In [10]:
#shape of the dataframe
df.shape

(103, 3)

In [11]:
df

,Postcode,Borough,Neighbourhood
0,M1E,Scarborough,"Guildwood,Morningside,West Hill"
1,M9M,North York,"Emery,Humberlea"
2,M5B,Downtown Toronto,"Ryerson,Garden District"
3,M1S,Scarborough,Agincourt
4,M1W,Scarborough,L'Amoreaux West
5,M5R,Central Toronto,"The Annex,North Midtown,Yorkville"
6,M3A,North York,Parkwoods
7,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
8,M2N,North York,Willowdale South
9,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
